<a href="https://colab.research.google.com/github/andreregino/llm-as-a-judge/blob/main/Step_B_LLM_as_a_Judge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import json

## Dataset

In [ ]:
# mounting the main folder from google drive

from os.path import join
from google.colab import drive

ROOT = "/content/drive"
drive.mount(ROOT)

%cd /content/drive/My Drive/CTI/Experimentos/llm-as-a-judge-e-commerce-triples/input/

Mounted at /content/drive
/content/drive/My Drive/CTI/Experimentos/llm-as-a-judge-e-commerce-triples/input


In [ ]:
csv_filename = "300_triples_multidomain_llama_fewshot.csv"

df = pd.read_csv(csv_filename)

df.head()

,summarized sentence,triple
0,O Central Multimídia Universal Mp5 Espelhament...,{central_multimidia_universal_mp5_espelhamento...
1,O Quadro Decorativo Grande Para Sala Quarto Le...,{quadro_decorativo_grande_para_sala_quarto_lea...
2,O Controle Ar Condicionado Universal Diversos ...,{controle_ar_condicionado_universal_diversos_m...
3,A Polia Correia Alternador Idea 2005/2010 4429...,{polia_correia_alternador_idea_2005_2010_4429}...
4,O Assoalho Fiat Uno 92 93 94 95 96 97 98 99 La...,{assoalho_fiat_uno_92_93_94_95_96_97_98_99_lad...


## Prompt LLM

In [ ]:
from google.colab import userdata

MARITACA_TOKEN = userdata.get('MARITACA_TOKEN')
GROQ_TOKEN = userdata.get('GROQ_FACCAMP_TOKEN')
TOGETHER_TOKEN = userdata.get('TOGETHER_TOKEN')

In [ ]:
import openai

def maritaca_query(content):
    client = openai.OpenAI(
        api_key=MARITACA_TOKEN,
        base_url="https://chat.maritaca.ai/api",
    )

    response = client.beta.chat.completions.parse(
      model="sabiazinho-3",
      messages=[
        {"role": "user", "content": content},
      ],
      response_format={"type": "json_object"},
      temperature=0.1
    )
    return response.choices[0].message.content


maritaca_query("Do you know how to build an RDF triple?")

'{"response": "Yes, to build an RDF (Resource Description Framework) triple, you need three components: a subject, a predicate, and an object. The subject is the resource being described, the predicate is a relationship or property that the subject has, and the object describes the value of that property or the resource to which the relationship refers. Here\'s an example of how to construct an RDF triple in a simple format: (Subject, Predicate, Object). For instance, if we were to represent the statement \'Socrates is a person\', it could be structured as follows in RDF triple format: (\'Socrates\', \'isA\', \'Person\')."}'

In [ ]:
import os
import openai

def groq_query(content, model_groq):
  client = openai.OpenAI(
      base_url="https://api.groq.com/openai/v1",
      api_key=GROQ_TOKEN
  )

  response = client.beta.chat.completions.parse(
      model=model_groq,
      messages=[
        {"role": "user", "content": content},
      ],
      response_format={"type": "json_object"},
      temperature=0.1
    )


  return response.choices[0].message.content

groq_query("Do you know how to build an RDF triple? Answer in json format", "openai/gpt-oss-120b")

'{"subject":"http://example.org/John","predicate":"http://xmlns.com/foaf/0.1/name","object":"John","turtle":"<http://example.org/John> <http://xmlns.com/foaf/0.1/name> \\"John\\" .","explanation":"An RDF triple is a statement consisting of a subject, a predicate, and an object, which can be expressed in Turtle syntax as shown."}'

In [ ]:
import os
import openai

def together_query(content, model_together):
  client = openai.OpenAI(
      base_url="https://api.together.xyz/v1",
      api_key=TOGETHER_TOKEN
  )

  response = client.chat.completions.create(
      model=model_together,
      messages=[
        {"role": "user", "content": content},
      ],
      response_format={"type": "json_object"},
      temperature=0.1,
      max_tokens=1500
    )


  return response.choices[0].message.content

together_query("Do you know how to build an RDF triple? Answer in json format. Be concise", "deepseek-ai/DeepSeek-V3")

'[  \n  {  \n    "subject": "URI or blank node",  \n    "predicate": "URI",  \n    "object": "URI, blank node, or literal"  \n  }  \n]'

In [ ]:
def judgment_prompt_few_shots():
  return '''
    Você é um especialista em ontologias e representação de conhecimento. Sua função é avaliar automaticamente a qualidade de triplas RDF geradas
    a partir de frases em linguagem natural. Você tem profundo conhecimento sobre estruturas semânticas,
    relações sujeito-predicado-objeto e boas práticas de modelagem de conhecimento.

    Dada uma frase descritiva e uma tripla RDF extraída automaticamente dessa frase, seu papel é atuar como um juiz avaliador,
    julgando a qualidade da tripla com base na fidelidade semântica e estrutural em relação à frase original.

    Avalie a tripla fornecida com base na frase original, respondendo às quatro primeiras perguntas com uma nota de 1 a 5 na escala de Likert,
    e justificando suas avaliações com um comentário de até 30 palavras.

    Etapas
    1. Leia atentamente a frase original.
    2. Analise a tripla RDF (sujeito, predicado, objeto).
    3. Avalie os seguintes aspectos com notas de 1 (inadequado/incorreto) a 5 (completamente adequado/correto):
    JUDGMENT 1: O sujeito está corretamente extraído?
    JUDGMENT 2: O predicado representa corretamente a relação expressa na frase?
    JUDGMENT 3: O objeto reflete corretamente o alvo da relação?
    JUDGMENT 4: A tripla representa com precisão a informação da frase?
    JUDGMENT 5: Justifique sua avaliação com uma frase de no máximo 30 palavras.

    Retorne a resposta em formato JSON, conforme os exemplos. Não é necessário retornar retornar os caracteres ``` e json, somente o objeto avaliação.

    Exemplos Few-Shot
    ----- ENTRADA -----
    SENTENÇA: O produto Kit Quadros... é disponível em tamanho maior, desde que seja solicitado o tamanho desejado.
    TRIPLA: {Kit Quadros...} {disponivel} {tamanho maior}

    ----- SAÍDA -----
    avaliacao {
        "JUDGMENT 1": 5,
        "JUDGMENT 2": 5,
        "JUDGMENT 3": 4,
        "JUDGMENT 4": 4,
        "JUDGMENT 5": "O sujeito e o predicado da tripla expressam fielmente o sujeito e a intenção da frase. No objeto, parece faltar a condição do tamanho desejado."
    }

    ----- ENTRADA -----
    SENTENÇA: Este suporte pode ser instalado em paredes de concreto ou madeira.
    TRIPLA: {suporte} {pode ser instalado}

    ----- SAÍDA -----
    avaliacao {
        "JUDGMENT 1": 5,
        "JUDGMENT 2": 4,
        "JUDGMENT 3": 1,
        "JUDGMENT 4": 2,
        "JUDGMENT 5": "A tripla está incompleta, pois omite completamente o alvo da instalação. Falta o objeto que especifica onde o suporte pode ser instalado."
    }
'''

def judgment_prompt_zero_shot():
  return '''
    Você é um especialista em ontologias e representação de conhecimento. Sua função é avaliar automaticamente a qualidade de triplas RDF geradas
    a partir de frases em linguagem natural. Você tem profundo conhecimento sobre estruturas semânticas,
    relações sujeito-predicado-objeto e boas práticas de modelagem de conhecimento.

    Dada uma frase descritiva e uma tripla RDF extraída automaticamente dessa frase, seu papel é atuar como um juiz avaliador,
    julgando a qualidade da tripla com base na fidelidade semântica e estrutural em relação à frase original.

    Avalie a tripla fornecida com base na frase original, respondendo às quatro primeiras perguntas com uma nota de 1 a 5 na escala de Likert,
    e justificando suas avaliações com um comentário de até 30 palavras.

    Etapas
    1. Leia atentamente a frase original.
    2. Analise a tripla RDF (sujeito, predicado, objeto).
    3. Avalie os seguintes aspectos com notas de 1 (inadequado/incorreto) a 5 (completamente adequado/correto):
    JUDGMENT 1: O sujeito está corretamente extraído?
    JUDGMENT 2: O predicado representa corretamente a relação expressa na frase?
    JUDGMENT 3: O objeto reflete corretamente o alvo da relação?
    JUDGMENT 4: A tripla representa com precisão a informação da frase?
    JUDGMENT 5: Justifique sua avaliação com uma frase de no máximo 30 palavras.

    Retorne a resposta em formato JSON, conforme os exemplos. Não é necessário retornar retornar os caracteres ``` e json, somente o objeto avaliação.

    ----- ENTRADA -----
    SENTENÇA: {SENTENÇA}
    TRIPLA: {SUJEITO} {PREDICADO} {OBJETO}

    ----- SAÍDA -----
    avaliacao {
        "JUDGMENT 1": {NOTA_1},
        "JUDGMENT 2": {NOTA_2},
        "JUDGMENT 3": {NOTA_3},
        "JUDGMENT 4": {NOTA_4},
        "JUDGMENT 5": "{JULGAMENTO}"
    }
'''


def build_prompt(sentence, triple):
  prompt = judgment_prompt_few_shots()
  prompt += "----- ENTRADA ------"
  prompt += "\nSENTENÇA: " + sentence
  prompt += "\nTRIPLA: " + triple
  prompt += "\n\n ----- SAIDA -----"
  return prompt


In [ ]:
judgments = []
for i, row in df.iterrows():
  #judgment = maritaca_query(build_prompt(row['summarized sentence'], row['triple']))
  judgment = groq_query(build_prompt(row['summarized sentence'], row['triple']), "openai/gpt-oss-120b")
  #judgment = huggingface_query(build_prompt(row['summarized sentence'], row['triple']))
  #judgment = together_query(build_prompt(row['summarized sentence'], row['triple']), "Qwen/Qwen2.5-7B-Instruct-Turbo")

  print(row['summarized sentence'])
  print(row['triple'])
  print(judgment)
  judgments.append(judgment)

O tamanho do Botas Segurança Calçado Pra Trabalho Epi C.a + Palmilha Gel é normal, correspondendo ao tamanho que o cliente está acostumado a usar.
{botas_seguancia_calcao_pra_trabalho_epi_c_a} {especificacao} {'tamanho normal'}
{
  "JUDGMENT 1": 3,
  "JUDGMENT 2": 2,
  "JUDGMENT 3": 5,
  "JUDGMENT 4": 3,
  "JUDGMENT 5": "Sujeito tem erros de ortografia e omite parte do nome; predicado genérico não reflete 'é normal'; objeto correto; tripla captura apenas parte da informação."
}
O Par Farol Milha Vectra 94 95 é vendido pelo valor referente ao par.
{par_farol_milha_vectra_94_95} {especificacao} {'vendido_por_par'}
{
  "JUDGMENT 1": 5,
  "JUDGMENT 2": 2,
  "JUDGMENT 3": 2,
  "JUDGMENT 4": 2,
  "JUDGMENT 5": "Sujeito correto, mas predicado e objeto não capturam a relação de venda/preço descrita na frase."
}
A cama box possui pés com altura de 12cm, e a base fica com 22cm de altura sem os pés.
{cama_box} {especificacao} {'altura 12cm e 22cm'}
{
  "JUDGMENT 1": 5,
  "JUDGMENT 2": 3,
  "JUDGM

In [ ]:
judgments

['{\n  "JUDGMENT 1": 3,\n  "JUDGMENT 2": 2,\n  "JUDGMENT 3": 5,\n  "JUDGMENT 4": 3,\n  "JUDGMENT 5": "Sujeito tem erros de ortografia e omite parte do nome; predicado genérico não reflete \'é normal\'; objeto correto; tripla captura apenas parte da informação."\n}',
 '{\n  "JUDGMENT 1": 5,\n  "JUDGMENT 2": 2,\n  "JUDGMENT 3": 2,\n  "JUDGMENT 4": 2,\n  "JUDGMENT 5": "Sujeito correto, mas predicado e objeto não capturam a relação de venda/preço descrita na frase."\n}',
 '{\n  "JUDGMENT 1": 5,\n  "JUDGMENT 2": 3,\n  "JUDGMENT 3": 2,\n  "JUDGMENT 4": 2,\n  "JUDGMENT 5": "Sujeito correto, mas predicado genérico e objeto combina duas alturas distintas sem diferenciar pés e base, perdendo detalhes da frase."\n}',
 '{"JUDGMENT 1":5,"JUDGMENT 2":4,"JUDGMENT 3":5,"JUDGMENT 4":4,"JUDGMENT 5":"Sujeito e objeto corretos; predicado genérico, mas transmite fato de ser ortopédico; falta detalhe da razão, porém informação essencial está presente."}',
 '{"JUDGMENT 1":5,"JUDGMENT 2":2,"JUDGMENT 3":2,"JUD

In [ ]:
import pandas as pd
import json
import re

def salvar_julgamentos_como_csv(lista_json, nome_arquivo):
    """
    Converte uma lista de strings JSON em um DataFrame e salva como CSV.

    Parâmetros:
    - lista_json (list): Lista de strings no formato JSON.
    - nome_arquivo (str): Nome do arquivo CSV de saída (com .csv no final).
    """
    registros = []

    for item in lista_json:
        # Remove blocos de markdown como ```json e ```
        item_limpo = re.sub(r"```|json|avaliacao", "", item, flags=re.IGNORECASE).strip()

        try:
            dados = json.loads(item_limpo)
            registros.append(dados)
        except json.JSONDecodeError as e:
            print(f"Erro ao decodificar o JSON: {e}\nConteúdo problemático: {item_limpo}\n")
            continue

    # Cria o DataFrame (sem forçar colunas específicas, para evitar erro se alguma estiver faltando)
    df = pd.DataFrame(registros)
    display(df)
    # Salva como CSV
    df.to_csv(nome_arquivo, index=False)
    print(f"Arquivo salvo como: {nome_arquivo}")


salvar_julgamentos_como_csv(judgments, "gpt120b_fs.csv")

,JUDGMENT 1,JUDGMENT 2,JUDGMENT 3,JUDGMENT 4,JUDGMENT 5
0,3,2,5,3,Sujeito tem erros de ortografia e omite parte ...
1,5,2,2,2,"Sujeito correto, mas predicado e objeto não ca..."
2,5,3,2,2,"Sujeito correto, mas predicado genérico e obje..."
3,5,4,5,4,"Sujeito e objeto corretos; predicado genérico,..."
4,5,2,2,2,"Sujeito correto, mas predicado genérico não re..."
...,...,...,...,...,...
110,5,4,5,4,Sujeito correto; predicado indica indisponibil...
111,5,4,5,4,Sujeito correto; predicado captura disponibili...
112,3,5,4,4,"Sujeito está parcialmente correto, omitindo de..."
113,5,4,5,4,Sujeito e objeto estão corretos; o predicado c...


Arquivo salvo como: gpt120b_fs.csv


## Evaluation of Judgment

In [ ]:
def evaluation_of_judgment_prompt_zero_shot():
  return '''
    Você é um avaliador especializado em análise metalinguística de julgamentos semânticos. Sua função é avaliar a qualidade do julgamento feito sobre a qualidade de uma tripla RDF extraída de uma frase.

    Você receberá:
    1. Uma sentença.
    2. Uma tripla RDF extraída dessa sentença.
    3. Um julgamento sobre essa tripla, contendo:
      - Quatro notas de 1 a 5 (escala de Likert), avaliando sujeito, predicado, objeto e fidelidade geral.
      - Uma justificativa textual curta explicando as notas atribuídas.

    Sua tarefa é analisar todos esses elementos e emitir **duas avaliações**, baseando-se exclusivamente nos dados apresentados:

    AVALIAÇÃO 1 — CLAREZA DA JUSTIFICATIVA: Quão clara e compreensível está a justificativa fornecida para as notas dadas?
    AVALIAÇÃO 2 — COERÊNCIA DAS NOTAS COM A JUSTIFICATIVA: As notas atribuídas parecem consistentes com a justificativa textual? Há alinhamento lógico?

    Ambas as avaliações devem ser expressas como notas de 1 a 5 na escala de Likert, sendo:
    - 1: Muito ruim
    - 2: Ruim
    - 3: Razoável
    - 4: Boa
    - 5: Excelente

    Retorne apenas um objeto JSON no seguinte formato:

    {
      "AVALIACAO 1": <nota de 1 a 5>,
      "AVALIACAO 2": <nota de 1 a 5>
    }

'''

def build_prompt_evaluation(sentence, triple, j1, j2, j3, j4, j5):
  prompt = evaluation_of_judgment_prompt_zero_shot()
  prompt += "----- ENTRADA ------"
  prompt += "\nSENTENÇA: " + sentence
  prompt += "\nTRIPLA: " + triple
  prompt += "\nJULGAMENTO 1, o sujeito foi corretamente extraido (nota de 1 a 5): " + str(j1)
  prompt += "\nJULGAMENTO 2, o predicado representa corretamente a relação expressa na frase (nota de 1 a 5): " + str(j2)
  prompt += "\nJULGAMENTO 3, o objeto reflete corretamente o alvo da relação (nota de 1 a 5): " + str(j3)
  prompt += "\nJULGAMENTO 4, a tripla representa com precisão a informação da frase (nota de 1 a 5): " + str(j4)
  prompt += "\nJULGAMENTO 5, justificativa da avaliação com uma frase curta: " + str(j5)
  prompt += "\n\n ----- SAIDA -----"
  return prompt


In [ ]:
arquivo = "gpt120b_fs.csv"
judgment_df = pd.read_csv("../output/"+arquivo)
display(judgment_df)

def obter_respostas_llm(df, judgment_df, modelo):
    respostas_brutas = []

    for i, row in df.iterrows():
        sentence = row['summarized sentence']
        triple = row['triple']

        julg = judgment_df.iloc[i]
        j1 = julg['JUDGMENT 1']
        j2 = julg['JUDGMENT 2']
        j3 = julg['JUDGMENT 3']
        j4 = julg['JUDGMENT 4']
        j5 = julg['JUDGMENT 5']

        resposta = together_query(build_prompt_evaluation(sentence, triple, j1, j2, j3, j4, j5), modelo)

        respostas_brutas.append({
            "sentence": sentence,
            "triple": triple,
            "JUDGMENT 1": j1,
            "JUDGMENT 2": j2,
            "JUDGMENT 3": j3,
            "JUDGMENT 4": j4,
            "JUDGMENT 5": j5,
            "resposta_bruta": resposta
        })

        display(triple)
        display(j1, j2, j3, j4)
        display(resposta)

    return respostas_brutas


,JUDGMENT 1,JUDGMENT 2,JUDGMENT 3,JUDGMENT 4,JUDGMENT 5
0,4,4,3,3,Sujeito está bem representado; predicado captu...
1,5,4,5,4,Sujeito está correto; predicado indica incompa...
2,5,5,5,5,"Sujeito, predicado e objeto correspondem fielm..."
3,5,5,5,5,"Sujeito, predicado e objeto correspondem exata..."
4,5,4,5,4,Sujeito e objeto estão corretos; o predicado c...
...,...,...,...,...,...
295,5,4,5,4,Sujeito correto; predicado indica indisponibil...
296,5,4,5,4,Sujeito correto; predicado captura disponibili...
297,3,5,4,4,"Sujeito está parcialmente correto, omitindo de..."
298,5,4,5,4,Sujeito e objeto estão corretos; o predicado c...


In [ ]:
import pandas as pd
import json

def processar_respostas_brutas(respostas_brutas):
    registros = []

    for i, item in enumerate(respostas_brutas):
        try:
            parsed = json.loads(item["resposta_bruta"])
            avaliacao_1 = parsed.get("AVALIACAO 1", None)
            avaliacao_2 = parsed.get("AVALIACAO 2", None)
        except Exception as e:
            print(f"[Erro na linha {i}] JSON inválido: {e}")
            avaliacao_1 = None
            avaliacao_2 = None

        registros.append({
            "sentence": item["sentence"],
            "triple": item["triple"],
            "JUDGMENT 1": item["JUDGMENT 1"],
            "JUDGMENT 2": item["JUDGMENT 2"],
            "JUDGMENT 3": item["JUDGMENT 3"],
            "JUDGMENT 4": item["JUDGMENT 4"],
            "JUDGMENT 5": item["JUDGMENT 5"],
            "resposta_bruta": item["resposta_bruta"],
            "AVALIACAO 1": avaliacao_1,
            "AVALIACAO 2": avaliacao_2
        })

    return pd.DataFrame(registros)


In [ ]:
respostas = obter_respostas_llm(df, judgment_df, "deepseek-ai/DeepSeek-V3")

df_avaliado = processar_respostas_brutas(respostas)

'{central_multimidia_universal_mp5_espelhamento_android_ios} {compativel} {meriva_2004}'

np.int64(4)

np.int64(4)

np.int64(3)

np.int64(3)

...

np.int64(5)

np.int64(4)

np.int64(4)

np.int64(4)

'{\n  "AVALIACAO 1": 5,\n  "AVALIACAO 2": 5\n}'

In [ ]:
df_avaliado.to_csv("julgamento_avaliacao_"+arquivo, index=False)

## Metrics

In [ ]:
import pandas as pd
import json

In [ ]:
# mounting the main folder from google drive

from os.path import join
from google.colab import drive

ROOT = "/content/drive"
drive.mount(ROOT)

%cd /content/drive/My Drive/CTI/Experimentos/llm-as-a-judge-e-commerce-triples/

Mounted at /content/drive
/content/drive/My Drive/CTI/Experimentos/llm-as-a-judge-e-commerce-triples


In [ ]:
csv_filename = "Resultados.csv"
df = pd.read_csv(csv_filename)

df.head()

,ID,Sentence,Triple,GS_SUBJECT,GS_PREDICATE,GS_OBJECT,GS_TRIPLE,GE_SM_ZS_SUBJECT,GE_SM_ZS_PREDICATE,GE_SM_ZS_OBJECT,...,GP_LA_ZS_EXPLAIN,GP_LA_ZS_EVALUATION_1,GP_LA_ZS_EVALUATION_2,GP_LA_FS_SUBJECT,GP_LA_FS_PREDICATE,GP_LA_FS_OBJECT,GP_LA_FS_TRIPLE,GP_LA_FS_EXPLAIN,GP_LA_FS_EVALUATION_1,GP_LA_FS_EVALUATION_2
0,1,O Central Multimídia Universal Mp5 Espelhament...,{central_multimidia_universal_mp5_espelhamento...,5,5,4,4,5,5,5,...,Sujeito e objeto corretos; predicado reflete r...,5.0,5.0,4,4,3,3,Sujeito está bem representado; predicado captu...,4,4
1,2,O Quadro Decorativo Grande Para Sala Quarto Le...,{quadro_decorativo_grande_para_sala_quarto_lea...,4,5,5,4,5,4,5,...,Sujeito correto; predicado genérico mas captur...,5.0,5.0,5,4,5,4,Sujeito está correto; predicado indica incompa...,5,5
2,3,O Controle Ar Condicionado Universal Diversos ...,{controle_ar_condicionado_universal_diversos_m...,5,5,5,5,5,5,4,...,Sujeito e objeto corretos; predicado indica co...,5.0,5.0,5,5,5,5,"Sujeito, predicado e objeto correspondem fielm...",5,5
3,4,A Polia Correia Alternador Idea 2005/2010 4429...,{polia_correia_alternador_idea_2005_2010_4429}...,5,5,5,5,5,5,5,...,Tripla captura corretamente que a polia não é ...,5.0,5.0,5,5,5,5,"Sujeito, predicado e objeto correspondem exata...",5,5
4,5,O Assoalho Fiat Uno 92 93 94 95 96 97 98 99 La...,{assoalho_fiat_uno_92_93_94_95_96_97_98_99_lad...,5,5,5,5,5,5,5,...,Sujeito e objeto corretos; predicado represent...,4.0,4.0,5,4,5,4,Sujeito e objeto estão corretos; o predicado c...,5,5


In [ ]:
import pandas as pd
from scipy.stats import kendalltau, spearmanr

def evaluate_llms(df, llm_names):
    results = []

    for llm in llm_names:
        row = {'LLM': llm}

        dimensions = ['SUBJECT', 'PREDICATE', 'OBJECT', 'TRIPLE']

        for dim in dimensions:
            gs_col = f"GS_{dim}"
            llm_col = f"{llm}_{dim}"

            gs_scores = df[gs_col].values
            llm_scores = df[llm_col].values

            # Compute Kendall's Tau and Spearman's Rho
            tau, p_value_kendal = kendalltau(gs_scores, llm_scores)
            rho, p_value_spearman = spearmanr(gs_scores, llm_scores)

            row[f"Kendall_Tau_{dim}"] = tau
            row[f"p-Kendall_Tau_{dim}"] = p_value_kendal
            row[f"Spearman_{dim}"] = rho
            row[f"p-Spearman_{dim}"] = p_value_spearman

        results.append(row)

    return pd.DataFrame(results)


In [ ]:
llms = [
  'GE_SM_ZS',
    'GE_SM_FS',
    'GE_LA_ZS',
    'GE_LA_FS',
    'QW_SM_ZS',
    'QW_SM_FS',
    'QW_LA_ZS',
    'QW_LA_FS',
    'SA_SM_ZS',
    'SA_SM_FS',
    'SA_LA_ZS',
    'SA_LA_FS',
    'GP_SM_ZS',
    'GP_SM_FS',
    'GP_LA_ZS',
    'GP_LA_FS'
]
df_results = evaluate_llms(df, llms)
display(df_results)


,LLM,Kendall_Tau_SUBJECT,p-Kendall_Tau_SUBJECT,Spearman_SUBJECT,p-Spearman_SUBJECT,Kendall_Tau_PREDICATE,p-Kendall_Tau_PREDICATE,Spearman_PREDICATE,p-Spearman_PREDICATE,Kendall_Tau_OBJECT,p-Kendall_Tau_OBJECT,Spearman_OBJECT,p-Spearman_OBJECT,Kendall_Tau_TRIPLE,p-Kendall_Tau_TRIPLE,Spearman_TRIPLE,p-Spearman_TRIPLE
0,GE_SM_ZS,0.106780,5.349913e-02,0.112000,5.263492e-02,0.199230,0.000181,0.215813,0.000165,0.314782,7.007744e-10,0.350279,4.361697e-10,0.334308,6.898848e-11,0.371826,2.864145e-11
1,GE_SM_FS,0.114782,3.952451e-02,0.119057,3.931731e-02,0.214187,0.000087,0.225476,0.000082,0.405399,1.777988e-14,0.434820,2.876289e-15,0.394738,1.287017e-14,0.434944,2.818739e-15
2,GE_LA_ZS,0.156885,4.430923e-03,0.164609,4.253308e-03,0.217140,0.000024,0.244443,0.000019,0.388353,3.937163e-14,0.428255,8.236969e-15,0.424329,4.566906e-17,0.479623,1.155536e-18
3,GE_LA_FS,0.129024,2.066067e-02,0.133830,2.040814e-02,0.183863,0.000622,0.198147,0.000557,0.382107,2.118270e-13,0.414457,6.989114e-14,0.394987,7.170084e-15,0.442838,7.711432e-16
4,QW_SM_ZS,0.070345,1.961060e-01,0.075177,1.941162e-01,0.101053,0.057736,0.110077,0.056856,-0.050228,3.370322e-01,-0.055044,3.420494e-01,0.050608,3.247294e-01,0.056505,3.293645e-01
5,QW_SM_FS,0.100274,7.174820e-02,0.104259,7.135692e-02,0.140427,0.011377,0.146590,0.011017,0.366319,8.348349e-13,0.405780,2.552289e-13,0.381587,4.513288e-14,0.425600,1.252467e-14
6,QW_LA_ZS,0.530017,2.259296e-22,0.559808,3.833310e-26,0.155023,0.002929,0.171522,0.002877,0.316451,8.978332e-10,0.350468,4.262618e-10,0.314437,3.154909e-10,0.357643,1.759560e-10
7,QW_LA_FS,0.241136,1.208635e-05,0.251775,1.014899e-05,0.167259,0.001498,0.182938,0.001461,0.272367,1.087123e-07,0.302922,8.741452e-08,0.353562,2.568883e-12,0.395768,1.086687e-12
8,SA_SM_ZS,0.128100,1.608319e-02,0.137294,1.734323e-02,0.143492,0.006534,0.156833,0.006490,0.353472,4.969861e-12,0.391741,1.919518e-12,0.422505,1.018974e-16,0.466536,1.278862e-17
9,SA_SM_FS,0.074879,1.792706e-01,0.077668,1.797064e-01,0.143401,0.010471,0.147628,0.010456,0.435765,1.102908e-15,0.459796,4.239723e-17,0.385815,3.116330e-13,0.420613,2.724897e-14


In [ ]:
df_results.to_csv('/content/drive/My Drive/CTI/Experimentos/llm-as-a-judge-e-commerce-triples/tau_spearman_results.csv', index=False)